In [ ]:
# app.py

import os
import streamlit as st
from fpdf import FPDF
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from typing import Optional, Type


def agentic_generate_script(topics: str, direction: str, speaker_style: str) -> str:
    """
    Generate a podcast script using an agentic approach with LangChain.
    The agent uses a zero-shot reaction style to process the prompt.
    """
    # Initialize the OpenAI LLM with your API key from Streamlit secrets
    llm = OpenAI(temperature=0.7, openai_api_key=st.secrets["OPENAI_API_KEY"])

    # Import and create a simple tool
    class ScriptGeneratorTool(BaseTool):
        name: str = "script_generator"
        description: str = (
            "Useful for generating podcast scripts based on topics and directions."
        )

        def _run(self, query: str) -> str:
            return f"Generated script based on: {query}"

        def _arun(self, query: str) -> str:
            return self._run(query)

    # Create a list with our tool
    tools = [ScriptGeneratorTool()]

    # Initialize the LangChain agent with the chosen AgentType
    agent = initialize_agent(
        tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )

    # Build the prompt that contains all necessary details
    prompt = f"""
    You are an AI podcast script generator. Your task is to produce a concise and engaging podcast script.
    Topics: {topics}
    Direction: {direction}
    Speaker style: {speaker_style}.
    Generate a script that is well-structured, engaging, and provides clear insights on these topics.
    """

    # Run the agent to generate the script
    script_result = agent.run(prompt)
    return script_result


def create_pdf(script_text: str, output_path: str) -> None:
    """
    Create a PDF file from the generated script using fpdf.
    """
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Split the script into lines and add them to the PDF
    for line in script_text.split("\n"):
        pdf.multi_cell(0, 10, line)
    pdf.output(output_path)


def main():
    st.title("Agentic AI Podcast Script Generator")

    # Input for topics (comma-separated if more than one)
    topics = st.text_input("Enter topic(s)", value="RAG, Fine-tuning, Vector Databases")

    # Input for podcast direction/instructions
    direction = st.text_area(
        "Enter instructions/direction for the podcast",
        value="Focus on explaining the relationship between these technologies and their impact on AI careers.",
    )

    # Dropdown for selecting the number of speakers
    speaker_style = st.selectbox("Number of speakers", ["One speaker", "Two speakers"])

    # When the user clicks the button, process inputs and generate PDF output
    if st.button("Generate Script & Save as PDF"):
        with st.spinner("Generating script..."):
            # Generate the podcast script using the agentic approach
            script_text = agentic_generate_script(topics, direction, speaker_style)

            # Ensure that the "scripts" folder exists
            if not os.path.exists("scripts"):
                os.makedirs("scripts")

            # Define the output path for the PDF
            pdf_file_path = os.path.join("scripts", "podcast_script.pdf")

            # Create and save the PDF file
            create_pdf(script_text, pdf_file_path)

        st.success("Done")
        st.write(
            "The generated podcast script has been saved as a PDF in the 'scripts' folder."
        )


if __name__ == "__main__":
    main()
